<a href="https://colab.research.google.com/github/Mehul852/chatbot1/blob/main/chatbot1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install torch
# pip install transformers

import sqlite3
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def create_database():
    conn = sqlite3.connect('user_details.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            mobile_number TEXT,
            email_id TEXT,
            date_of_birth TEXT
        )
    ''')
    conn.commit()
    conn.close()

create_database()


def gather_user_details():
    user_details = {}
    user_details['name'] = input('Enter your name: ')
    user_details['mobile_number'] = input('Enter your mobile number: ')
    user_details['email_id'] = input('Enter your email ID: ')
    user_details['date_of_birth'] = input('Enter your date of birth (DD/MM/YYYY): ')

    conn = sqlite3.connect('user_details.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO users (name, mobile_number, email_id, date_of_birth)
        VALUES (:name, :mobile_number, :email_id, :date_of_birth)
    ''', user_details)
    conn.commit()
    conn.close()

    return user_details


model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def get_response(input_text, chat_history_ids):
    new_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response, chat_history_ids

user_details = gather_user_details()
print(f"User Details: {user_details}")

chat_history_ids = None
while True:
    request = input('You: ')
    if request.lower() == 'ok':
        print('Bot: Bye!')
        break
    else:
        response, chat_history_ids = get_response(request, chat_history_ids)
        print('Bot:', response)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Enter your name: mehul
Enter your mobile number: 7978615078
Enter your email ID: mehul.bholu570@gmail.com
Enter your date of birth (DD/MM/YYYY): 08/03/1999
User Details: {'name': 'mehul', 'mobile_number': '7978615078', 'email_id': 'mehul.bholu570@gmail.com', 'date_of_birth': '08/03/1999'}
You: hi
Bot: Hi
You: ok
Bot: Bye!
